## MEDIUM MODEL

Medium model based on KNN

In [2]:
import pandas as pd
import json
import ast
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [3]:
# Load the dataset

df = pd.read_csv('../../data/final/books_translated.csv')
df

,title,series,author,rating,description,language,genres,bookFormat,pages,publisher,publishDate,firstPublishDate,awards,coverImg,series_number,desc_length
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,en,"['adventure', 'dystopia', 'fantasy']",Hardcover,374,Scholastic Press,2008-09-14 00:00:00,False,True,https://i.gr-assets.com/images/S/compressed.ph...,1.0,147
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary Grandpré (Illustrator)",4.50,There is a door at the end of a silent corrido...,en,"['adventure', 'childrens', 'classics']",Paperback,870,Scholastic Inc.,2004-09-28 00:00:00,True,True,https://i.gr-assets.com/images/S/compressed.ph...,5.0,136
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,en,"['classics', 'fiction', 'historical']",Paperback,324,Harper Perennial Modern Classics,2006-05-23 00:00:00,True,True,https://i.gr-assets.com/images/S/compressed.ph...,NaN,137
3,Pride and Prejudice,Standalone Novel,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,en,"['classics', 'fiction', 'historical']",Paperback,279,Modern Library,2000-10-10 00:00:00,True,False,https://i.gr-assets.com/images/S/compressed.ph...,NaN,100
4,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,en,"['fantasy', 'fiction', 'paranormal']",Paperback,501,"Little, Brown and Company",2006-09-06 00:00:00,True,True,https://i.gr-assets.com/images/S/compressed.ph...,1.0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57388,Beasts & Behemoths (Dungeons & Dragons),Standalone Novel,"Jim Zub, Stacy King, Andrew Wheeler, Official ...",3.60,Study this guide and keep it close at hand--th...,en,['fiction'],Paperback,114,Ten Speed Press,2020-10-20 00:00:00,True,False,http://books.google.com/books/content?id=1toui...,NaN,170
57389,Faculty of Dragon Riders,Standalone Novel,Dmitry Nazarov,3.60,I tamed the Black Dragon!So I thought until I ...,en,['fiction'],Paperback,401,Litres,2022-08-24 00:00:00,True,False,http://books.google.com/books/content?id=QSGFE...,NaN,52
57390,Midnight Delivery Sex,Standalone Novel,Neneko Narazaki,4.00,With SNS card for collecting in the first edit...,de,['comics'],Paperback,29,Hayabusa,2021-05-04 00:00:00,True,False,http://books.google.com/books/content?id=s_8_E...,NaN,78
57391,Monster Girl: 2,Standalone Novel,Kazuki Funatsu,3.50,"Dopo il loro incontro, Yatsuki si ritrova ad a...",it,"['comics', 'isekai']",Paperback,216,Edizioni BD,2020-05-01 00:00:00,True,False,http://books.google.com/books/content?id=_yjnE...,NaN,48


In [4]:
# Normalize the titles to ensure consistent matching

df['normalized_title'] = df['title'].str.strip().str.lower()

In [5]:
# One-hot encode the genres

mlb = MultiLabelBinarizer()
genre_ohe = mlb.fit_transform(df['genres'])

In [6]:
# Create a new dataframe with the one-hot encoded genres

genre_df = pd.DataFrame(genre_ohe, columns=mlb.classes_, index=df.index)
genre_df
# Concatenate the one-hot encoded genres with the original dataframe

df = pd.concat([df, genre_df], axis=1)
df

,title,series,author,rating,description,language,genres,bookFormat,pages,publisher,...,m,n,o,p,r,s,t,u,v,y
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,en,"['adventure', 'dystopia', 'fantasy']",Hardcover,374,Scholastic Press,...,0,1,1,1,1,1,1,1,1,1
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary Grandpré (Illustrator)",4.50,There is a door at the end of a silent corrido...,en,"['adventure', 'childrens', 'classics']",Paperback,870,Scholastic Inc.,...,0,1,0,0,1,1,1,1,1,0
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,en,"['classics', 'fiction', 'historical']",Paperback,324,Harper Perennial Modern Classics,...,0,1,1,0,1,1,1,0,0,0
3,Pride and Prejudice,Standalone Novel,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,en,"['classics', 'fiction', 'historical']",Paperback,279,Modern Library,...,0,1,1,0,1,1,1,0,0,0
4,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,en,"['fantasy', 'fiction', 'paranormal']",Paperback,501,"Little, Brown and Company",...,1,1,1,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57388,Beasts & Behemoths (Dungeons & Dragons),Standalone Novel,"Jim Zub, Stacy King, Andrew Wheeler, Official ...",3.60,Study this guide and keep it close at hand--th...,en,['fiction'],Paperback,114,Ten Speed Press,...,0,1,1,0,0,0,1,0,0,0
57389,Faculty of Dragon Riders,Standalone Novel,Dmitry Nazarov,3.60,I tamed the Black Dragon!So I thought until I ...,en,['fiction'],Paperback,401,Litres,...,0,1,1,0,0,0,1,0,0,0
57390,Midnight Delivery Sex,Standalone Novel,Neneko Narazaki,4.00,With SNS card for collecting in the first edit...,de,['comics'],Paperback,29,Hayabusa,...,1,0,1,0,0,1,0,0,0,0
57391,Monster Girl: 2,Standalone Novel,Kazuki Funatsu,3.50,"Dopo il loro incontro, Yatsuki si ritrova ad a...",it,"['comics', 'isekai']",Paperback,216,Edizioni BD,...,1,0,1,0,0,1,0,0,0,0


In [9]:
# TRAIN THE KNN MODEL WITH THE ONE-HOT ENCODED GENRES:

X = genre_df

model = NearestNeighbors(n_neighbors=10, metric='cosine')
model.fit(X)

,n_neighbors,10
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [10]:
# IMPROVEMENT OF THE RECOMMENDATION SYSTEM

def recommend_by_genre(title, exclude_same_series=True, exclude_same_author=False):
    # Normalize the title entered by the user
    normalized_title = title.strip().lower()

    if normalized_title not in df['normalized_title'].values:
        return f"❌ Book '{title}' not found on the dataset."

    # Look for the book on the DataSet index
    idx = df[df['normalized_title'] == normalized_title].index[0]
    book = df.loc[idx]

    # Get the checked book's vector
    book_vector = X.iloc[idx].values.reshape(1, -1)

    # Obtain the KNN
    distances, indices = model.kneighbors(book_vector)

    # Start the recommendations list
    recommendations = []

    for i in range(1, len(indices[0])):  # Skip the same book (index 0)
        candidate_idx = indices[0][i]
        candidate = df.iloc[candidate_idx]

        #  OPTIONAL BOOK EXCLUSION OF THE SAME SERIE
        if exclude_same_series:
            if (candidate['series'] == book['series']) and (candidate['series'] != 'missing'):
                continue
            # Also exclusion of the books with similar keywords (e.g.: Harry Potter)
            if book['title'].split()[0] in candidate['title']:
                continue

        # OPTIONAL EXCLUSION OF SAME AUTHORS BOOKS
        if exclude_same_author:
            if candidate['author'] == book['author']:
                continue

        recommendations.append(candidate)

    # Show recommendations
    print(f"\n📚 Recommendations for: \033[1m{book['title']}\033[0m\n")
    if not recommendations:
        print("No recommendations were found with the filters applied..")
        return

    for rec in recommendations[:5]:  # Only show 5 recommendations
        print(f"- {rec['title']} ({rec['author']}) | Genres: {', '.join(rec['genres'])}")

In [ ]:
# EXAMPLE USAGE

recommend_by_genre("Harry Potter and the sorcerer's stone")


📚 Recommendations for: Harry Potter and the Sorcerer's Stone

- Watership Down (Richard Adams) | Genres: [, ', a, d, v, e, n, t, u, r, e, ', ,,  , ', c, h, i, l, d, r, e, n, s, ', ,,  , ', c, l, a, s, s, i, c, s, ', ]
- The Sky Phantom (Carolyn Keene) | Genres: [, ', a, d, v, e, n, t, u, r, e, ', ,,  , ', c, h, i, l, d, r, e, n, s, ', ,,  , ', c, l, a, s, s, i, c, s, ', ]
- Little Town in the Ozarks (Roger Lea MacBride, David Gilleece (Illustrations)) | Genres: [, ', a, d, v, e, n, t, u, r, e, ', ,,  , ', c, h, i, l, d, r, e, n, s, ', ,,  , ', c, l, a, s, s, i, c, s, ', ]
- East of the Sun & West of the Moon (Mercer Mayer) | Genres: [, ', a, d, v, e, n, t, u, r, e, ', ,,  , ', c, h, i, l, d, r, e, n, s, ', ,,  , ', c, l, a, s, s, i, c, s, ', ]
- Caboose Mystery (Gertrude Chandler Warner) | Genres: [, ', a, d, v, e, n, t, u, r, e, ', ,,  , ', c, h, i, l, d, r, e, n, s, ', ,,  , ', c, l, a, s, s, i, c, s, ', ]


c:\Users\noemi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
